# Prediction heart disease using machine learning 

This notebook showcases an end to end machine learning framework using data on heart disease from the University of California Irvine. 

## Problem definition

In a statement, 
> Given clinical parameters about a patient, can we predict whether or not a person has a heart disease?
